## Ananya's Music App

Objective: An app where users can specify what they are in the mood for and the app generates a Spotify playlist (max 15 songs) based on the user's top tracks + top releases from their favorite artists input.

The user will be asked to make selections either from a list of options or on a scale (1-5) for the following:
<ul>
    <li><b>What mood are you in?</b> - Users can choose from 'Good Mood' or 'Bad Mood' [Valence] </li>
    <li><b>Do you want to dance?</b> - Scale of 1-5 (1: no dancing, 5 very dancing) [Danceability]</li>
    <li><b>What activity best describes the scene?</b> - Selection: Solo Chill, Hanging with Friends, Social Gathering, Party, Rager [Energy] </li>
    <li><b>What genre are you interested in?</b> - [TBD; will be selection based]

## Installations, Imports, & Credentials

In [ ]:
#Install necessary Libraries
# pip install spotipy
# pip install streamlit

In [ ]:
#Needed to run Streamlit app in their cloud service
#pip freeze > requirements.txt

In [66]:
#Imports
import os
import pandas as pd
import streamlit as st
from collections import Counter
from spotipy import SpotifyException
import time
import requests
from IPython.display import display, Markdown
import requests

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

In [8]:
#Set Environment Variables to prevent Harcoding adn Retrieve with the below code
# client_id = os.getenv('SPOTIPY_CLIENT_ID')
# client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')


In [9]:
port = int(os.environ.get('PORT', 9000))
redirect_uri = f'http://localhost:{port}/callback'

#Hardcoded
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='<INSERT ID>',
                                               client_secret='<INSERT SECRET>',
                                               redirect_uri=redirect_uri,
                                               scope='user-read-private user-top-read playlist-modify-private'))

## Defining a Universe of Tracks

First, we will define the scope of our universe. The tracks we will use to generate playlists will be a user's top 50 most listened to songs in the 'medium term' + top 10 songs released by artists in their top 50 (if not already in their list).

### Retrieving a user's top 50 tracks

In [11]:
#Current User Data - this will launch a window that will prompt you to log in to Spotify 
#and you should copy/paste the URL you are directed to in the box that appears below
user = sp.current_user()
user

{'display_name': 'Ananya Tinaikar',
 'external_urls': {'spotify': 'https://open.spotify.com/user/1275022654'},
 'href': 'https://api.spotify.com/v1/users/1275022654',
 'id': '1275022654',
 'images': [],
 'type': 'user',
 'uri': 'spotify:user:1275022654',
 'followers': {'href': None, 'total': 56},
 'country': 'US',
 'product': 'premium',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False}}

In [12]:
username = user['display_name']
userlink = user['external_urls']['spotify']
userid = user['id']

print(f'We are generating a playlist for {username}, who can be found at {userlink}')

We are generating a playlist for Ananya Tinaikar, who can be found at https://open.spotify.com/user/1275022654


### Functions to Flatten Response and Retrieve Genres

In [13]:
def get_attributes(row):
    
    release_dt = row['album'].get('release_date')
    
    artistinfo = []  
    
    for artist in row['album'].get('artists'):
        
        name = artist.get('name')
        id = artist.get('id')
        uri = artist.get('uri')
        
        artistinfo.append([name, id, uri])
    
    return [artistinfo, release_dt]
        

def top_track_response(response):

    if 'items' in response.keys():
        tracks = pd.DataFrame(response['items'])
    else:
        tracks = pd.DataFrame(response['tracks'])

    tracks['flat'] = tracks.apply(get_attributes,axis=1)


    return tracks

In [14]:
def get_artist_genres(artist_ids):  
    
    artists_info = pd.DataFrame()
    
    # Split artist IDs into batches of 50 (maximum allowed by Spotify API)
    batch_size = 50
    
    for i in range(0, len(artist_ids), batch_size):
        batch_ids = artist_ids[i:i + batch_size]
        response = sp.artists(batch_ids)['artists']
        response_df = pd.DataFrame(response)
        
        # Get artist information for the batch of IDs
        artists_info = pd.concat([artists_info,response_df])
        
            
    return artists_info

In [15]:
user_toptrack = sp.current_user_top_tracks(time_range='medium_term', limit=50)
your_tops = top_track_response(user_toptrack)

In [16]:
your_tops['artists_flat'] = your_tops['flat'].str[0]
your_tops['release_date'] = your_tops['flat'].str[1]

#your_tops = your_tops.set_index('id')
your_tops = your_tops.drop(columns=['flat'])
your_tops['artists_names'] = your_tops['artists_flat'].apply(lambda x: [artist[0] for artist in x])
your_tops['artists_ids'] = your_tops['artists_flat'].apply(lambda x: [artist[1] for artist in x])

In [17]:
your_tops = your_tops.explode(['artists_names','artists_ids'])
your_tops = your_tops.reset_index(drop=True)

In [18]:
#get_artist_genres(your_tops['artists_ids'].tolist())
artist_ids = your_tops['artists_ids'].tolist()
genres = get_artist_genres(artist_ids)

In [19]:
genres = genres[['id','genres']]
genres = genres.drop_duplicates(subset='id', keep='first')
genres = genres.reset_index(drop=True)

In [20]:
your_tops = pd.merge(your_tops, genres, left_on='artists_ids', right_on='id', how='outer')
your_tops = your_tops.drop(columns=['id_y']).rename(columns={'id_x':'id'})

In [21]:
your_tops

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,...,popularity,preview_url,track_number,type,uri,artists_flat,release_date,artists_names,artists_ids,genres
0,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,186040,True,{'isrc': 'QZPEW2438198'},{'spotify': 'https://open.spotify.com/track/5E...,https://api.spotify.com/v1/tracks/5EpQ3S8Lvqz4...,5EpQ3S8Lvqz4juCGv4NXX8,...,11,https://p.scdn.co/mp3-preview/9565dfbfc8394831...,1,track,spotify:track:5EpQ3S8Lvqz4juCGv4NXX8,"[[Myles Cameron, 2B2dZ6tZkA7MAHcPaHw1wF, spoti...",2024-05-07,Myles Cameron,2B2dZ6tZkA7MAHcPaHw1wF,[chill r&b]
1,"{'album_type': 'SINGLE', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,213133,False,{'isrc': 'QZVEM2388797'},{'spotify': 'https://open.spotify.com/track/6O...,https://api.spotify.com/v1/tracks/6OvLJA8LYyDN...,6OvLJA8LYyDNtm9vrBWGEY,...,21,https://p.scdn.co/mp3-preview/85ea92ee576b3a2c...,1,track,spotify:track:6OvLJA8LYyDNtm9vrBWGEY,"[[Myles Cameron, 2B2dZ6tZkA7MAHcPaHw1wF, spoti...",2024-01-30,Myles Cameron,2B2dZ6tZkA7MAHcPaHw1wF,[chill r&b]
2,"{'album_type': 'SINGLE', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,157533,True,{'isrc': 'QZVEM2363057'},{'spotify': 'https://open.spotify.com/track/2k...,https://api.spotify.com/v1/tracks/2kWa0CB8BPIp...,2kWa0CB8BPIptCR53wRJTN,...,2,https://p.scdn.co/mp3-preview/2b5f34dd633bf712...,2,track,spotify:track:2kWa0CB8BPIptCR53wRJTN,"[[Myles Cameron, 2B2dZ6tZkA7MAHcPaHw1wF, spoti...",2024-01-30,Myles Cameron,2B2dZ6tZkA7MAHcPaHw1wF,[chill r&b]
3,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,180866,False,{'isrc': 'QZPEW2438200'},{'spotify': 'https://open.spotify.com/track/18...,https://api.spotify.com/v1/tracks/18klQHQroXxx...,18klQHQroXxxtAMZRlALUL,...,12,https://p.scdn.co/mp3-preview/c81851624ab1d8e3...,4,track,spotify:track:18klQHQroXxxtAMZRlALUL,"[[Myles Cameron, 2B2dZ6tZkA7MAHcPaHw1wF, spoti...",2024-05-07,Myles Cameron,2B2dZ6tZkA7MAHcPaHw1wF,[chill r&b]
4,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,236286,False,{'isrc': 'INS180808433'},{'spotify': 'https://open.spotify.com/track/0E...,https://api.spotify.com/v1/tracks/0ErXZ1NMqOEF...,0ErXZ1NMqOEFGJPQvU9OKF,...,53,https://p.scdn.co/mp3-preview/3db023c99eb78c8e...,2,track,spotify:track:0ErXZ1NMqOEFGJPQvU9OKF,"[[Salim–Sulaiman, 6ohaQzKaXrobAL8paLSaxq, spot...",2008-09-19,Salim–Sulaiman,6ohaQzKaXrobAL8paLSaxq,"[filmi, indian folk, modern bollywood]"
5,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,327508,False,{'isrc': 'GBAHS2400149'},{'spotify': 'https://open.spotify.com/track/39...,https://api.spotify.com/v1/tracks/397GpgLAJnwQ...,397GpgLAJnwQDBwIJ8ko1t,...,59,https://p.scdn.co/mp3-preview/314337ec242ff6e7...,1,track,spotify:track:397GpgLAJnwQDBwIJ8ko1t,"[[Fred again.., 4oLeXFyACqeem2VImYeBFe, spotif...",2024-04-09,Fred again..,4oLeXFyACqeem2VImYeBFe,"[edm, house, stutter house]"
6,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,250291,False,{'isrc': 'GBAHS2100378'},{'spotify': 'https://open.spotify.com/track/1u...,https://api.spotify.com/v1/tracks/1uyQNwG1sFl7...,1uyQNwG1sFl7etjFTEHlQp,...,53,https://p.scdn.co/mp3-preview/4c9c69e2cc265779...,9,track,spotify:track:1uyQNwG1sFl7etjFTEHlQp,"[[Fred again.., 4oLeXFyACqeem2VImYeBFe, spotif...",2021-11-19,Fred again..,4oLeXFyACqeem2VImYeBFe,"[edm, house, stutter house]"

### Top 10 Songs from Each Artist in Our Top 50

In [22]:
# batch_size = 50
artist_toptracks = pd.DataFrame()

for i in range(0, len(your_tops['artists_ids'])):
    
    toptracks = sp.artist_top_tracks(your_tops['artists_ids'][i])
    toptracks = top_track_response(toptracks).reset_index(drop=True)
    artist_toptracks = pd.concat([artist_toptracks,toptracks],axis=0)
    
artist_toptracks = artist_toptracks.reset_index(drop=True)

In [23]:
artist_toptracks['artists_flat'] = artist_toptracks['flat'].str[0]
artist_toptracks['release_date'] = artist_toptracks['flat'].str[1]

In [24]:
artist_toptracks = artist_toptracks.drop(columns=['flat'])
artist_toptracks['artists_names'] = artist_toptracks['artists_flat'].apply(lambda x: [artist[0] for artist in x])
artist_toptracks['artists_ids'] = artist_toptracks['artists_flat'].apply(lambda x: [artist[1] for artist in x])

In [25]:
artist_toptracks = artist_toptracks.explode(['artists_names','artists_ids'])
artist_toptracks = artist_toptracks.reset_index(drop=True)

In [26]:
artist_tops_ids = artist_toptracks['artists_ids'].tolist()
genres = get_artist_genres(artist_tops_ids)
genres = genres[['id','genres']]
genres = genres.drop_duplicates(subset='id', keep='first')

In [27]:
artist_toptracks = artist_toptracks.merge(genres, left_on='artists_ids',right_on='id',how='outer')
artist_toptracks = artist_toptracks.drop(columns=['id_y']).rename(columns={'id_x':'id'}).reset_index(drop=True)

In [28]:
#Final Universe of Tracks to Generate Playlist From
playlist_universe = pd.concat([your_tops,artist_toptracks],ignore_index=True)
playlist_universe

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,...,preview_url,track_number,type,uri,artists_flat,release_date,artists_names,artists_ids,genres,is_playable
0,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,186040,True,{'isrc': 'QZPEW2438198'},{'spotify': 'https://open.spotify.com/track/5E...,https://api.spotify.com/v1/tracks/5EpQ3S8Lvqz4...,5EpQ3S8Lvqz4juCGv4NXX8,...,https://p.scdn.co/mp3-preview/9565dfbfc8394831...,1,track,spotify:track:5EpQ3S8Lvqz4juCGv4NXX8,"[[Myles Cameron, 2B2dZ6tZkA7MAHcPaHw1wF, spoti...",2024-05-07,Myles Cameron,2B2dZ6tZkA7MAHcPaHw1wF,[chill r&b],NaN
1,"{'album_type': 'SINGLE', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,213133,False,{'isrc': 'QZVEM2388797'},{'spotify': 'https://open.spotify.com/track/6O...,https://api.spotify.com/v1/tracks/6OvLJA8LYyDN...,6OvLJA8LYyDNtm9vrBWGEY,...,https://p.scdn.co/mp3-preview/85ea92ee576b3a2c...,1,track,spotify:track:6OvLJA8LYyDNtm9vrBWGEY,"[[Myles Cameron, 2B2dZ6tZkA7MAHcPaHw1wF, spoti...",2024-01-30,Myles Cameron,2B2dZ6tZkA7MAHcPaHw1wF,[chill r&b],NaN
2,"{'album_type': 'SINGLE', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,157533,True,{'isrc': 'QZVEM2363057'},{'spotify': 'https://open.spotify.com/track/2k...,https://api.spotify.com/v1/tracks/2kWa0CB8BPIp...,2kWa0CB8BPIptCR53wRJTN,...,https://p.scdn.co/mp3-preview/2b5f34dd633bf712...,2,track,spotify:track:2kWa0CB8BPIptCR53wRJTN,"[[Myles Cameron, 2B2dZ6tZkA7MAHcPaHw1wF, spoti...",2024-01-30,Myles Cameron,2B2dZ6tZkA7MAHcPaHw1wF,[chill r&b],NaN
3,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,180866,False,{'isrc': 'QZPEW2438200'},{'spotify': 'https://open.spotify.com/track/18...,https://api.spotify.com/v1/tracks/18klQHQroXxx...,18klQHQroXxxtAMZRlALUL,...,https://p.scdn.co/mp3-preview/c81851624ab1d8e3...,4,track,spotify:track:18klQHQroXxxtAMZRlALUL,"[[Myles Cameron, 2B2dZ6tZkA7MAHcPaHw1wF, spoti...",2024-05-07,Myles Cameron,2B2dZ6tZkA7MAHcPaHw1wF,[chill r&b],NaN
4,"{'album_type': 'ALBUM', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,236286,False,{'isrc': 'INS180808433'},{'spotify': 'https://open.spotify.com/track/0E...,https://api.spotify.com/v1/tracks/0ErXZ1NMqOEF...,0ErXZ1NMqOEFGJPQvU9OKF,...,https://p.scdn.co/mp3-preview/3db023c99eb78c8e...,2,track,spotify:track:0ErXZ1NMqOEFGJPQvU9OKF,"[[Salim–Sulaiman, 6ohaQzKaXrobAL8paLSaxq, spot...",2008-09-19,Salim–Sulaiman,6ohaQzKaXrobAL8paLSaxq,"[filmi, indian folk, modern bollywood]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,NaN,1,128438,False,{'isrc': 'GBKPL2205059'},{'spotify': 'https://open.spotify.com/track/7K...,https://api.spotify.com/v1/tracks/7KnGAY27aFqg...,7KnGAY27aFqgiYySwt7yg5,...,https://p.scdn.co/mp3-preview/ede56dcbbfdd73bf...,7,track,spotify:track:7KnGAY27aFqgiYySwt7yg5,"[[Djo, 5p9HO3XC5P3BLxJs5Mtrhm, spotify:artist:...",2022-09-16,Djo,5p9HO3XC5P3BLxJs5Mtrhm,"[pov: indie, psychedelic pop]",True
729,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,NaN,1,184314,False,{'isrc': 'GBKPL2205063'},{'spotify': 'https://open.spotify.com/track/6T...,https://api.spotify.com/v1/tracks/6TorufIR9E06...,6TorufIR9E06RXo3VhZaae,...,https://p.scdn.co/mp3-preview/a50c71ae801b7b7a...,12,track,spotify:track:6TorufIR9E06RXo3VhZaae,"[[Djo, 5p9HO3XC5P3BLxJs5Mtrhm, spotify:artist:...",2022-09-16,Djo,5p9HO3XC5P3BLxJs5Mtrhm,"[pov: indie, psychedelic pop]",True
730,"

In [29]:
# # Define aggregation functions
# def unique_names(x):
#     unique_names_set = set(x)
#     return ', '.join(unique_names_set)

aggregations = {
    #'id': lambda x: ', '.join(set(', '.join(x).split(', '))),
    'artists_ids': lambda x: ', '.join(x),
    'genres': 'sum',
    'artists_names': lambda x: ', '.join(map(str, pd.Series(x).drop_duplicates()))
}

unique_universe = playlist_universe.groupby('id').agg(aggregations).reset_index()

In [30]:
unique_universe

,id,artists_ids,genres,artists_names
0,022ps5TymdGgPGus4ojM4u,7Ez6lTtSMjMf2YSYpukP1I,[],Dasha
1,02rdXe0KhMe8p6ZHzYtuw0,5SXuuuRpukkTvsLuUknva1,"[hip hop, rap]",Baby Keem
2,04mAOoQNsXmDJlyupJwmkO,0LyfQWJT6nXafLPZqxe9Of,[],Various Artists
3,05jEfIK1V6ELyqz87oUUnN,3oTDO2HdnIHS42f8E6KIFJ,[],DJ Max Star
4,084ERnS6WhLay9MrRLo7Hw,"2B2dZ6tZkA7MAHcPaHw1wF, 2B2dZ6tZkA7MAHcPaHw1wF...","[chill r&b, chill r&b, chill r&b, chill r&b]",Myles Cameron
...,...,...,...,...
465,7uA5gUTAB85bUXDBLb1LBy,"3csPCeXsj2wezyvkRFzvmV, 0sElgscu7tp38PM1MtsUz7","[big beat, breakbeat, electronica, intelligent...","Orbital, Energy 52"
466,7wZZoaI2AykMAL4nyYMKv3,0jW6R8CVyVohuUJVcuweDI,[bow pop],The Piano Guys
467,7xi6IzpwkgtpenLzl5LMLc,0QJIPDAEDILuo8AIq3pMuU,"[escape room, indietronica, new rave]",M.I.A.
468,7xoUc6faLbCqZO6fQEYprd,66CXWjxzNUsdJxJ2JdwvnR,[pop],Ariana Grande


## Retrieve Audio Features for Tracks in our Universe

Now that we have our playlist universe, we get "audio features" for each of our tracks. Audio features we are interested in include:
<ol>
    <li>Danceability - Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. </li>
    <li>Energy - Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity.</li>
    <li>Valence - How positive the song sounds</li> 
</ol>

In [31]:
def get_audio_features(track_ids):
 
    dance, energy, valence = [], [], []
    
    batch_size = 100
    
    for i in range(0,len(track_ids),batch_size):
        
        batch_ids = list(track_ids[i:i+batch_size])
        aud_feat = sp.audio_features(batch_ids)
        
        for feat in aud_feat:
            if feat is not None:
                dance.append(feat['danceability'])
                energy.append(feat['energy'])
                valence.append(feat['danceability'])
            else:
                dance.append(0)
                energy.append(0)
                valence.append(0)
    
    print('Done!')
    return dance, energy, valence

In [32]:
tracks = list(unique_universe['id'])

unique_universe['danceability'], unique_universe['energy'], unique_universe['valence'] = get_audio_features(tracks)

Done!


## Filtering Playlist

Next we will define a function to filter the playlist based on user-specified inputs for three questions (each corresponding to a particular audio feature)

In [33]:
#Function to Filter Playlist based on Selections
def filter_playlist(df,valence,dance,activity):
    
    mapping = {} 
    
    if valence:
        mapping = {
            'Bad': (0.0,0.4999),
            'Good': (0.5,1.0)
        }
        val_min, val_max = mapping.get(valence, (0.0,1.0))
        
    if dance:
        mapping = {
             1: (0.0, 0.2),
             2: (0.2, 0.4),
             3: (0.4, 0.6),
             4: (0.6, 0.8),
             5: (0.8, 1.0)
        } 
        dance_min, dance_max = mapping.get(dance, (0.0,1.0))
    
    if activity:
        mapping = {
             'Solo Chill': (0.0, 0.2),
             'Chill with Friends': (0.2, 0.4),
             'Social Gathering': (0.4, 0.6),
             'Party': (0.6, 0.8),
             'Rager': (0.8, 1.0)
        } 
        act_min, act_max = mapping.get(activity, (0.0,1.0))
    
    
    df = df[(df['valence'] > val_min) & (df['valence'] < val_max)]
    df = df[(df['danceability'] > dance_min) & (df['danceability'] < dance_max)]
    df = df[(df['energy'] > act_min) & (df['energy'] < act_max)]
    df = df.reset_index(drop=True)

    return df

In [34]:
val_value = input('What mood are you in? ')
dan_value = input('How much do you want to dance? ')
act_value = input('What activity are you doing? ')

What mood are you in? Good
How much do you want to dance? 5
What activity are you doing? Rager


In [35]:
result_playlist = filter_playlist(unique_universe,val_value,dan_value,act_value)
result_playlist

,id,artists_ids,genres,artists_names,danceability,energy,valence
0,0HMqknanFIn0efQ5U4nUHT,74D1UgRzMhTSPz698exXmR,"[bhangra, classic bhangra, desi hip hop]",Panjabi MC,0.719,0.806,0.719
1,0M9E0hfrH44KPVX6AJoVnn,7hQmAXAzWI6D350VTgkKTG,[persian pop],Arash,0.708,0.904,0.708
2,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,"[dance pop, hip hop, hip pop, neo soul, pop ra...",Missy Elliott,0.904,0.813,0.904
3,0bHnzW0bxdghfzdX0BhDht,1JdHoZwkwOmX2Pl2iTwnzc,"[filmi, modern bollywood]",Vishal Bhardwaj,0.817,0.922,0.817
4,0cVeP9t3QChKYHKNGWfPnD,0QJIPDAEDILuo8AIq3pMuU,"[escape room, indietronica, new rave]",M.I.A.,0.582,0.844,0.582
...,...,...,...,...,...,...,...
109,7lPYsLXUUZMhWACOLi9U86,"61pcWUzNqpHNWc21SuHP1J, 75xNYf2GU5wtQqBrd74SlY","[escape room, brisbane indie, dark trap]","The Buttress, Zheani",0.504,0.944,0.504
110,7rCrOTI49IE199dlDkGcC5,1q4618qKswelCGLoanFKQh,[rap rock],Joey Valence & Brae,0.808,0.987,0.808
111,7skteOnDbwZ3ZvjSe99xv2,0PCCGZ0wGLizHt2KZ7hhA2,[],Artemas,0.656,0.876,0.656
112,7uA5gUTAB85bUXDBLb1LBy,"3csPCeXsj2wezyvkRFzvmV, 0sElgscu7tp38PM1MtsUz7","[big beat, breakbeat, electronica, intelligent...","Orbital, Energy 52",0.606,0.933,0.606


## Logic to Ensure 15 songs per Playlist

We are defining a good playlist as one that has 15 songs. However, there are cases where filtering parameters may result in a final universe that is < 15 songs. We will define methodology to address such cases.

In [36]:
flat_genres = []

for sublist in result_playlist['genres']:
    flat_genres.extend(sublist)

flat_genres = list(set(flat_genres))
flat_genres

['neo-psychedelic',
 'europop',
 'belgian dance',
 'brass band',
 'canadian pop',
 'madchester',
 'electro',
 'second line',
 'pop rap',
 'german techno',
 'indietronica',
 'nu disco',
 'australian dance',
 'virginia hip hop',
 'trance',
 'dance rock',
 'brisbane indie',
 'big beat',
 'pop',
 'german hip hop',
 'new romantic',
 'alternative hip hop',
 'filmi',
 'new rave',
 'german trance',
 'permanent wave',
 'complextro',
 'rap',
 'desi hip hop',
 'escape room',
 'new wave pop',
 'house',
 'hip pop',
 'electropop',
 'indie pop rap',
 'aussietronica',
 'australian electropop',
 'indie electropop',
 'indie soul',
 'alternative r&b',
 'edm',
 'desi pop',
 'tamil hip hop',
 'modern rock',
 'rock',
 'pop edm',
 'classic bollywood',
 'bubble trance',
 'new wave',
 'australian psych',
 'progressive house',
 'neo soul',
 'rap rock',
 'bhangra',
 'vapor soul',
 'breakbeat',
 'alabama rap',
 'r&b',
 'alternative dance',
 'experimental r&b',
 'pov: indie',
 'big room',
 'post-punk',
 'brazilian

In [37]:
#Define Function to Ensure 15 song Playlist
def fifteen_songs(unique_universe,result_playlist):
    #Retrieving list of current unique genres in dataset
    flat_genres = []

    for sublist in result_playlist['genres']:
        flat_genres.extend(sublist)

    flat_genres = list(set(flat_genres)) #Storing as variable
    
    r_index = 0 #row index
    l_index = 0 #
    filter_genre = flat_genres[l_index] #current genre we are filtering for
    uni_parsed = len(unique_universe)#number of rows to parse through

    #Checking if we've parsed through all rows yet 
    while uni_parsed >= 0:
        for genres in unique_universe['genres']:
            uni_parsed -= 1
            r_index += 1
            for item in genres:
                if item==filter_genre:
                    row = pd.DataFrame(unique_universe.iloc[r_index]).transpose()
                    result_playlist = pd.concat([result_playlist,row],ignore_index=True)
                    break
            #Checking length of result playlist/uni parsed
            if len(result_playlist) >= 15:
                uni_parsed = -1
                break
        break

    return result_playlist

In [38]:
result = fifteen_songs(unique_universe,result_playlist)
result

,id,artists_ids,genres,artists_names,danceability,energy,valence
0,0HMqknanFIn0efQ5U4nUHT,74D1UgRzMhTSPz698exXmR,"[bhangra, classic bhangra, desi hip hop]",Panjabi MC,0.719,0.806,0.719
1,0M9E0hfrH44KPVX6AJoVnn,7hQmAXAzWI6D350VTgkKTG,[persian pop],Arash,0.708,0.904,0.708
2,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,"[dance pop, hip hop, hip pop, neo soul, pop ra...",Missy Elliott,0.904,0.813,0.904
3,0bHnzW0bxdghfzdX0BhDht,1JdHoZwkwOmX2Pl2iTwnzc,"[filmi, modern bollywood]",Vishal Bhardwaj,0.817,0.922,0.817
4,0cVeP9t3QChKYHKNGWfPnD,0QJIPDAEDILuo8AIq3pMuU,"[escape room, indietronica, new rave]",M.I.A.,0.582,0.844,0.582
...,...,...,...,...,...,...,...
109,7lPYsLXUUZMhWACOLi9U86,"61pcWUzNqpHNWc21SuHP1J, 75xNYf2GU5wtQqBrd74SlY","[escape room, brisbane indie, dark trap]","The Buttress, Zheani",0.504,0.944,0.504
110,7rCrOTI49IE199dlDkGcC5,1q4618qKswelCGLoanFKQh,[rap rock],Joey Valence & Brae,0.808,0.987,0.808
111,7skteOnDbwZ3ZvjSe99xv2,0PCCGZ0wGLizHt2KZ7hhA2,[],Artemas,0.656,0.876,0.656
112,7uA5gUTAB85bUXDBLb1LBy,"3csPCeXsj2wezyvkRFzvmV, 0sElgscu7tp38PM1MtsUz7","[big beat, breakbeat, electronica, intelligent...","Orbital, Energy 52",0.606,0.933,0.606


## Map the Song IDs to their Names

In [39]:
song_name_map = playlist_universe[['id','name']].drop_duplicates()
song_name_map

,id,name
0,5EpQ3S8Lvqz4juCGv4NXX8,Boyhood
1,6OvLJA8LYyDNtm9vrBWGEY,Luh Ya
2,2kWa0CB8BPIptCR53wRJTN,Softboy
3,18klQHQroXxxtAMZRlALUL,Sundown
4,0ErXZ1NMqOEFGJPQvU9OKF,Mar Jawaan
...,...,...
728,7KnGAY27aFqgiYySwt7yg5,I Want Your Video
729,6TorufIR9E06RXo3VhZaae,Figure You Out
730,56sDw3DIGcUStBO5SguYPK,Fool
731,3HAYmp6l5lYGG8zz4qMbIX,Personal Lies


In [57]:
result = result.merge(song_name_map, on='id',how='inner')

In [58]:
#Spotify has a limit of # of songs that can be written to a playlsit
if len(result) > 100:
    result = result[0:100]

## Create Playlist via API

In [59]:
songs = list(result['id'])

In [60]:
def write_playlist(dataframe):
    songs = list(dataframe['id'])
    
    playlist = sp.user_playlist_create(user=userid, name='By Pyotr',public=False)
    playlist = sp.playlist_add_items(playlist_id=playlist['id'], items=songs)
    
    if playlist:
        print("Pyotr has written your playlist to your account. Do svidaniya")

In [62]:
write_playlist(result)

Pyotr has written your playlist to your account. Do svidaniya


In [69]:
import sys
print(sys.executable)

C:\Anaconda\python.exe
